In [7]:
import json
import math
import os
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from copy import deepcopy
from collections import Counter, defaultdict
%matplotlib inline

## Part 1

In [8]:
move_costs = {
    'A': 1,
    'B': 10,
    'C': 100,
    'D': 1000,
}
home_rooms = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
}

In [9]:
def print_game(rooms, hallway):
    num_rooms = len(rooms)
    hallway_len = len(hallway)
    room_size = len(rooms[0])
    
    print('#' * (hallway_len + 2))
    print('#', end='')
    for piece in hallway:
        if piece is None:
            print('.', end='')
        else:
            print(piece[0], end='')
    print('#')
    for r in range(room_size - 1, -1, -1):
        char = '#' if r == room_size - 1 else ' '
        print(char * ((hallway_len + 2 - (2 * num_rooms) - 1) // 2), end='')
        for i in range(num_rooms):
            if rooms[i][r] is not None:
                piece = rooms[i][r][0]
            else:
                piece = '.'
            print(f'#{piece}', end='')
        print('#', end='')
        print(char * ((hallway_len + 2 - (2 * num_rooms) - 1) // 2))
    print(' ' * ((hallway_len + 2 - (2 * num_rooms) - 1) // 2), end='')
    for i in range(len(rooms)):
        print('##', end='')
    print('#', end='')
    print(' ' * ((hallway_len + 2 - (2 * num_rooms) - 1) // 2))

In [40]:
def get_moves(piece, rooms, hallway, doors):
    debug = False

    piece_type = piece[0]
    home_room = home_rooms[piece_type]
    move_cost = move_costs[piece_type]
    room_size = len(rooms[0])

    if piece in hallway:
        loc = 'hallway'
    else:
        loc = 'room'

    moves = []
    if loc == 'room':
        # In a room
        for i, room in enumerate(rooms):
            if piece in room:
                room_number = i
                room_index = room.index(piece)
        if room_index < (room_size - 1) and sum(r is not None for r in rooms[room_number][room_index+1:]) > 0:
            # Blocked in room
            if debug: print('blocked in room')
            return []
        if room_number == home_room:
            # In home room
            if sum(r is not None and r[0] != piece_type for r in rooms[home_room]) == 0:
                # With only other home pieces
                if debug: print('already in home room')
                return []
        # Move into hallway
        room_door = doors[room_number]
        for i in range(room_door + 1, len(hallway)):
            if hallway[i] is None:
                if i not in doors:
                    cost = ((room_size - room_index) + (i - room_door)) * move_cost
                    moves.append(('hallway', i, cost))
            else:
                break
        for i in range(room_door - 1, -1, -1):
            if hallway[i] is None:
                if i not in doors:
                    cost = ((room_size - room_index) + (room_door - i)) * move_cost
                    moves.append(('hallway', i, cost))
            else:
                break
        return moves
        # Ignoring direct moves to other rooms for simplicity
    else:
        # In hallway
        if rooms[home_room][room_size - 1] is not None:
            # Home room full already
            if debug: print('home room already full')
            return []
        if sum(r is not None and r[0] != piece_type for r in rooms[home_room]) > 0:
            # Home room has a different piece in it
            if debug: print('home room has different piece in it')
            return []
        cur_pos = hallway.index(piece)
        if cur_pos < doors[home_room]:
            if sum(hallway[i] is not None for i in range(cur_pos + 1, doors[home_room] + 1)) > 0:
                # Blocked path to home room
                if debug: print('path to home room blocked')
                return []
        else:
            if sum(hallway[i] is not None for i in range(doors[home_room], cur_pos)) > 0:
                # Blocked path to home room
                if debug: print('path to home room blocked')
                return []
        if cur_pos > doors[home_room]:
            cost = (cur_pos - doors[home_room]) * move_cost
        else:
            cost = (doors[home_room] - cur_pos) * move_cost
        space = min(r for r, space in enumerate(rooms[home_room]) if space is None)
        cost += (room_size - space) * move_cost
        moves.append(('room', (home_room, space), cost))
        return moves

In [41]:
def do_move(piece, move, rooms, hallway):
    new_rooms = deepcopy(rooms)
    new_hallway = deepcopy(hallway)
    if move[0] == 'room':
        # Move to room
        new_hallway[new_hallway.index(piece)] = None
        new_rooms[move[1][0]][move[1][1]] = piece
    else:
        # Move to hallway
        for i, room in enumerate(rooms):
            if piece in room:
                room_number = i
                room_index = room.index(piece)
        new_rooms[room_number][room_index] = None
        new_hallway[move[1]] = piece
    return new_rooms, new_hallway

In [42]:
def get_all_moves(rooms, hallway, doors):
    moves = []
    all_pieces = [''.join(p) for p in itertools.product(move_costs.keys(), [str(v + 1) for v in range(len(rooms[0]))])]
    for piece in all_pieces:
        piece_moves = get_moves(piece, rooms, hallway, doors)
        complete_piece_moves = []
        for move in piece_moves:
            complete_piece_moves.append((piece, *move))
        if len(complete_piece_moves) > 0:
            moves.extend(complete_piece_moves)
    return moves

In [43]:
def game_finished(rooms, hallway, doors):
    if sum(h is not None for h in hallway) == 0:
        wrong_room = False
        for piece, i in home_rooms.items():
            for r in range(len(rooms[0])):
                if rooms[i][r][0] != piece:
                    return False
        # Every piece is home
        return True
    return False

In [44]:
def game_stalled(rooms, hallway, doors):
    return len(get_all_moves(rooms, hallway, doors)) == 0

In [32]:
def explore_moves(rooms, hallway, doors, cost_so_far):
    global evaluations, best_so_far
    if game_finished(rooms, hallway, doors):
        if cost_so_far < best_so_far:
            best_so_far = cost_so_far
            print(best_so_far)
        return 0, []
    min_cost = 1e100
    evaluations += 1
    all_moves = sorted(get_all_moves(rooms, hallway, doors), key=lambda m: m[3])
    
    room_moves = [move for move in all_moves if move[1] == 'room']
    if len(room_moves) > 0:
        all_moves = [room_moves[0]]
    
    for move in all_moves:
        piece, ma, mb, move_cost = move
        if cost_so_far + move_cost > best_so_far:
            continue
        new_rooms, new_hallway = do_move(piece, (ma, mb), rooms, hallway)
        if game_stalled(new_rooms, new_hallway, doors) and not game_finished(new_rooms, new_hallway, doors):
            continue
        resulting_cost, new_moves = explore_moves(new_rooms, new_hallway, doors, cost_so_far + move_cost)
        if resulting_cost == -1:
            continue
        if move_cost + resulting_cost < min_cost:
            min_cost = move_cost + resulting_cost
            moves = [move] + new_moves
    if min_cost == 1e100:
        return -1, []
    return min_cost, moves

In [33]:
doors = [2, 4, 6, 8]

In [46]:
# Example
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########

test_rooms = [
    ['A1', 'B1'],
    ['D1', 'C1'],
    ['C2', 'B2'],
    ['A2', 'D2'],
]
test_hallway = [None, None, None, None, None, None, None, None, None, None, None]
print_game(test_rooms, test_hallway)

#############
#...........#
###B#C#B#D###
  #A#D#C#A#  
  #########  


In [45]:
# Input
#############
#...........#
###C#C#B#D###
  #D#A#B#A#
  #########

test_rooms = [
    ['D1', 'C1'],
    ['A1', 'C2'],
    ['B1', 'B2'],
    ['A2', 'D2'],
]
test_hallway = [None, None, None, None, None, None, None, None, None, None, None]
print_game(test_rooms, test_hallway)

#############
#...........#
###C#C#B#D###
  #D#A#B#A#  
  #########  


In [47]:
evaluations = 0
best_so_far = 1e100
print(explore_moves(test_rooms, test_hallway, doors, 0))
print(evaluations)

12581
12563
12541
12539
12521
(12521, [('B2', 'hallway', 3, 40), ('C1', 'hallway', 5, 200), ('C1', 'room', (2, 1), 200), ('D2', 'hallway', 7, 2000), ('A2', 'hallway', 9, 3), ('D2', 'room', (3, 0), 3000), ('D1', 'hallway', 5, 3000), ('B2', 'room', (1, 0), 30), ('D1', 'room', (3, 1), 4000), ('B1', 'hallway', 3, 20), ('B1', 'room', (1, 1), 20), ('A2', 'room', (0, 1), 8)])
402324


In [48]:
best_so_far

12521

## Part 2

In [61]:
# Input
#############
#...........#
###C#C#B#D###
  #D#C#B#A#
  #D#B#A#C#
  #D#A#B#A#
  #########

test_rooms = [
    ['D1', 'D2', 'D3', 'C1'],
    ['A1', 'B1', 'C2', 'C3'],
    ['B2', 'A2', 'B3', 'B4'],
    ['A3', 'C4', 'A4', 'D4'],
]
test_hallway = [None, None, None, None, None, None, None, None, None, None, None]
print_game(test_rooms, test_hallway)

#############
#...........#
###C#C#B#D###
  #D#C#B#A#  
  #D#B#A#C#  
  #D#A#B#A#  
  #########  


In [59]:
# Example
#############
#...........#
###B#C#B#D###
  #D#C#B#A#
  #D#B#A#C#
  #A#D#C#A#
  #########

test_rooms = [
    ['A1', 'D1', 'D2', 'B1'],
    ['D3', 'B2', 'C1', 'C2'],
    ['C3', 'A2', 'B3', 'B4'],
    ['A3', 'C4', 'A4', 'D4'],
]
test_hallway = [None, None, None, None, None, None, None, None, None, None, None]
print_game(test_rooms, test_hallway)

#############
#...........#
###B#C#B#D###
  #D#C#B#A#  
  #D#B#A#C#  
  #A#D#C#A#  
  #########  


In [57]:
# Example Test
#############
#AA.D.B.B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#.#C#C#
  #A#.#C#A#
  #########

test_rooms = [
    ['A3', 'D3', 'D4', 'B4'],
    [None, None, None, None],
    ['C1', 'C2', 'C3', None],
    ['A4', 'C4', None, None],
]
test_hallway = ['A1', 'A2', None, 'D1', None, 'B1', None, 'B2', None, 'B3', 'D2']
print_game(test_rooms, test_hallway)

#############
#AA.D.B.B.BD#
###B#.#.#.###
  #D#.#C#.#  
  #D#.#C#C#  
  #A#.#C#A#  
  #########  


In [62]:
evaluations = 0
best_so_far = 1e100
print(explore_moves(test_rooms, test_hallway, doors, 0))
print(evaluations)

47393
47193
(47193, [('B4', 'hallway', 10, 50), ('B3', 'hallway', 9, 50), ('A2', 'hallway', 0, 9), ('B2', 'hallway', 7, 50), ('C1', 'hallway', 3, 200), ('C1', 'room', (2, 0), 700), ('C3', 'hallway', 5, 200), ('C3', 'room', (2, 1), 400), ('C2', 'hallway', 5, 300), ('C2', 'room', (2, 2), 300), ('B1', 'hallway', 5, 40), ('A1', 'hallway', 1, 7), ('B1', 'room', (1, 0), 50), ('B2', 'room', (1, 1), 60), ('B3', 'room', (1, 2), 70), ('B4', 'room', (1, 3), 70), ('D3', 'hallway', 3, 3000), ('D4', 'hallway', 5, 4000), ('A4', 'hallway', 10, 4), ('C4', 'hallway', 7, 400), ('C4', 'room', (2, 3), 200), ('A3', 'hallway', 9, 5), ('D4', 'room', (3, 0), 7000), ('D3', 'room', (3, 1), 8000), ('D2', 'hallway', 3, 4000), ('D2', 'room', (3, 2), 7000), ('D1', 'hallway', 3, 5000), ('A1', 'room', (0, 0), 5), ('A2', 'room', (0, 1), 5), ('D1', 'room', (3, 3), 6000), ('A3', 'room', (0, 2), 9), ('A4', 'room', (0, 3), 9)])
5767915
